In [1]:
from os import listdir
from itertools import combinations
import random
import pandas as pd
import cv2
from numpy import expand_dims
from keras.models import load_model
import pickle
import numpy as np
import keras

Using TensorFlow backend.


In [2]:
with open("face_embeddings.pkl", 'rb') as file:
    embd_dict = pickle.load(file)
    
df = pd.read_csv("data.csv")
df.head()

,image1,image2,label
0,data/Billy_Crystal_0001.jpg,data/Billy_Crystal_0003.jpg,1
1,data/Britney_Spears_0008.jpg,data/Britney_Spears_0014.jpg,1
2,data/Joseph_Deiss_0002.jpg,data/Joseph_Deiss_0003.jpg,1
3,data/Barbra_Streisand_0001.jpg,data/Barbra_Streisand_0002.jpg,1
4,data/Rebekah_Chantay_Revels_0001.jpg,data/Rebekah_Chantay_Revels_0002.jpg,1


In [48]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, df, embd_dict, embd_size, batch_size=8, shuffle=True):
        'Initialization'
        
        self.batch_size = batch_size
        self.df = df
        self.list_IDs = list_IDs
        self.embd_dict = embd_dict
        self.embd_size = embd_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_batch):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X1 = np.empty((self.batch_size, self.embd_size))
        X2 = np.empty((self.batch_size, self.embd_size))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_batch):
            # Store sample
            img1 = self.df['image1'].iloc[ID]
            img2 = self.df['image2'].iloc[ID]
            X1[i,] = self.embd_dict[img1]
            X2[i,] = self.embd_dict[img2]

            # Store class
            y[i] = self.df['label'].iloc[ID]

        return [X1, X2], y

In [49]:
from sklearn.model_selection import train_test_split
train_idx, val_idx = train_test_split(
    df.index, random_state=42, test_size=0.2, stratify = df.label.values)


In [50]:
train_face_gen = DataGenerator(train_idx, df, embd_dict, embd_size=128)
val_face_gen = DataGenerator(val_idx, df, embd_dict, embd_size=128)

In [39]:
for x,y in train_face_gen:
    pass
for x,y in val_face_gen:
    pass

In [51]:
from keras.layers import Dense,Flatten,Input,Layer
from keras.models import Model
import keras.backend as K
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Convolution2D
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import merge, Input, Lambda
from keras.models import Model
from keras.layers.noise import GaussianNoise
from keras.utils import generic_utils
from keras.optimizers import RMSprop, Adam

def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

def base():
    xin = Input(shape=(128,))    
    x1 = GaussianNoise(0.5)(xin)
    x1 = Dropout(0.2)(x1)
    x1 = Dense(128)(x1)
    x1 = BatchNormalization()(x1)
    x1 = LeakyReLU(0.33)(x1)
    x1 = Dropout(0.2)(x1)
    
    return Model(xin, x1)

def face_detector():
    base_model = base()
    
    x1 = Input(shape=(128,)) 
    x2 = Input(shape=(128,)) 
    embd1 = base_model(x1)
    embd2 = base_model(x2)
    
    distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([embd1, embd2])
    
    face_detector = Model(input = [x1, x2], output = distance)
    return face_detector

In [54]:
model = face_detector()
adam = RMSprop()
model.compile(loss=contrastive_loss, optimizer=adam, metrics=[accuracy])
model.summary()

Model: "model_23"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_36 (InputLayer)           (None, 128)          0                                            
__________________________________________________________________________________________________
input_37 (InputLayer)           (None, 128)          0                                            
__________________________________________________________________________________________________
model_22 (Model)                (None, 128)          17024       input_36[0][0]                   
                                                                 input_37[0][0]                   
__________________________________________________________________________________________________
lambda_13 (Lambda)              (None, 1)            0           model_22[1][0]            

C:\Users\adishjain01\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:62: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("la...)`


In [55]:
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler

def lr_control(epoch):
    if epoch < 20:
        lr = 1e-3
    elif epoch < 40:
        lr = 3e-4
    elif epoch >= 40:
        lr = 2e-5
    return lr

callback = [ModelCheckpoint(f'best_model_saved_embd_conloss.h5', save_best_only=True),
            EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=20, restore_best_weights=True),
            #ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, min_lr=0.00000001, mode='max', verbose=1),
           LearningRateScheduler(lr_control)]

history = model.fit_generator(
        train_face_gen,
        validation_data=val_face_gen,
        epochs=50,
        verbose=2,
    callbacks=callback
    )

Epoch 1/50
 - 2s - loss: 42.5504 - accuracy: 0.5000 - val_loss: 2.7697 - val_accuracy: 0.5000
Epoch 2/50
 - 1s - loss: 23.9082 - accuracy: 0.5000 - val_loss: 2.8095 - val_accuracy: 0.5000
Epoch 3/50
 - 1s - loss: 12.1299 - accuracy: 0.5000 - val_loss: 1.6148 - val_accuracy: 0.5000
Epoch 4/50
 - 1s - loss: 5.4742 - accuracy: 0.5000 - val_loss: 0.4362 - val_accuracy: 0.5050
Epoch 5/50
 - 1s - loss: 2.4409 - accuracy: 0.5000 - val_loss: 0.0992 - val_accuracy: 0.6025
Epoch 6/50
 - 1s - loss: 1.0957 - accuracy: 0.5000 - val_loss: 0.1183 - val_accuracy: 0.9750
Epoch 7/50
 - 1s - loss: 0.3765 - accuracy: 0.5000 - val_loss: 0.2817 - val_accuracy: 0.5000
Epoch 8/50
 - 0s - loss: 0.2341 - accuracy: 0.5544 - val_loss: 0.1941 - val_accuracy: 0.5075
Epoch 9/50
 - 1s - loss: 0.2107 - accuracy: 0.6744 - val_loss: 0.2298 - val_accuracy: 0.5725
Epoch 10/50
 - 1s - loss: 0.1996 - accuracy: 0.7331 - val_loss: 0.2828 - val_accuracy: 0.6450
Epoch 11/50
 - 1s - loss: 0.1953 - accuracy: 0.7394 - val_loss: 0.